In [ ]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_2.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_3.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_4.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_5.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_nomove_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_varspeed_1.npz', allow_pickle=False)

# measurement = np.load('../../dataset/meas_asymm_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_2.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_nomove_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_reflector_1.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

trainData - (6400, 15, 251, 2)
testData - (1600, 15, 251, 2)


In [2]:
def get64Samples(cirs):

    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    
    # Number of signalsd
    num_signals = real.shape[0]  # 3 in this case
    # print(f'num_signals: {num_signals}')
    # Initialize lists to store the focused samples
    imp_real_parts = []
    imp_imag_parts = []
    img_mag_parts = []
    
    for i in range(num_signals):
        # Calculate the magnitude
        magnitude = np.abs(real[i] + 1j * imag[i])
        
        # find the peak index
        peak_index = np.argmax(magnitude)
        
        # Calculate the start and end indices for the focused part
        start_index = max(0, peak_index - 32)
        end_index = min(magnitude.shape[0], peak_index + 32)
        
        # Extract the part of the signal around the peak
        real_part_focus = real[i, start_index:end_index]
        imag_part_focus = imag[i, start_index:end_index]
        mag_part_focus = magnitude[start_index:end_index]
        
        imp_real_parts.append(real_part_focus)
        imp_imag_parts.append(imag_part_focus)
        img_mag_parts.append(mag_part_focus)
        

    # Convert lists back to arrays for further processing if needed
    imp_real_parts = np.array(imp_real_parts)
    imp_imag_parts = np.array(imp_imag_parts)
    img_mag_parts = np.array(img_mag_parts)

    return imp_real_parts, imp_imag_parts, img_mag_parts

In [3]:
def getRealImaginaryParts(cirs):
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    return real, imag

In [ ]:
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Extract data for ALICE and BOB channels
alice_train_CIRs = trainCIR[:, alice_channel, :, :]  
eve_train_CIRs = trainCIR[:, eve_channel, :, :] # (100, 251, 2)
print(f'alice_train_CIRs : {alice_train_CIRs.shape}')

# Feature extraction
alice_train_real, alice_train_imag = getRealImaginaryParts(alice_train_CIRs)
alice_train_features = np.hstack((alice_train_real, alice_train_imag))


eve_train_real, eve_train_imag = getRealImaginaryParts(eve_train_CIRs)
eve_train_features = np.hstack((eve_train_real, eve_train_imag))


# Create labels for Alice and Eve for training
alice_train_labels = np.zeros(alice_train_features.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(eve_train_features.shape[0])       # Label '1' for Eve.


# Combine data and labels for training
train_atoms = np.vstack((alice_train_features, eve_train_features))
train_labels = np.hstack((alice_train_labels, eve_train_labels))

# Apply PCA for Dimensionality Reduction
scaler = StandardScaler()
train_atoms_normalized = scaler.fit_transform(train_atoms)
print(f'atoms normalized : {train_atoms_normalized.shape}')

n_components = 250
pca = PCA(n_components=250)  # please check the number of components like 2 or 3
train_atoms_pca = pca.fit_transform(train_atoms_normalized)
print(f'atoms normalized pca : {train_atoms_pca.shape}')

D = train_atoms_pca.T

# explained_variance = pca.explained_variance_ratio_
# print(f"Explained variance by each component: {explained_variance}")
# print(f"Total explained variance by {n_components} components: {np.sum(explained_variance)}")

alice_train_CIRs : (6400, 251, 2)
atoms normalized : (12800, 502)
atoms normalized pca : (12800, 250)
Explained variance by each component: [0.1684378  0.16186283 0.01398361 0.01356117 0.01102857 0.01089184
 0.00965671 0.00929481 0.00903283 0.00875903 0.00846603 0.00817402
 0.00776928 0.00761858 0.00747248 0.00740464 0.00728212 0.00701868
 0.00684711 0.00675448 0.00674316 0.00665213 0.00645379 0.00637682
 0.00630317 0.00623279 0.00615875 0.00603792 0.00591664 0.00584711
 0.00568224 0.00559761 0.00554654 0.00551471 0.00548356 0.00538374
 0.00529309 0.00520206 0.00517553 0.00505779 0.00493955 0.00490225
 0.00486517 0.00475522 0.00471402 0.00458181 0.00456747 0.00451988
 0.00447848 0.00443019 0.00439658 0.00432703 0.00429987 0.00422662
 0.00417009 0.00410085 0.00405299 0.00400057 0.00391995 0.00383698
 0.00382178 0.00378389 0.00373898 0.00368817 0.0036308  0.00356349
 0.00354518 0.0035024  0.00342438 0.00340164 0.00331154 0.00329398
 0.00327601 0.00321183 0.00319792 0.00314659 0.0030814  

In [5]:
# Step 3: Extract Features for Test Data
alice_test_CIRs = testCIR[:, alice_channel, :, :]
eve_test_CIRs = testCIR[:, eve_channel, :, :]


alice_test_real, alice_test_imag = getRealImaginaryParts(alice_test_CIRs)
alice_test_features = np.hstack((alice_test_real, alice_test_imag))

eve_test_real, eve_test_imag = getRealImaginaryParts(eve_test_CIRs)
eve_test_features = np.hstack((eve_test_real, eve_test_imag))

# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(alice_test_features.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_features.shape[0])       # Label '1' for Eve.

# Combine data and labels for testing
test_atoms = np.vstack((alice_test_features, eve_test_features))
test_labels = np.hstack((alice_test_labels, eve_test_labels))

test_atoms_normalized = scaler.transform(test_atoms)
test_atoms_pca = pca.transform(test_atoms_normalized)

In [6]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=15):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    # Predict the class with the smallest residual
    # Find the index of the smallest residual
    min_residual_index = np.argmin(residuals)
    # Use this index to find the corresponding class
    predicted_class = unique_classes[min_residual_index]
    
    # print(f'predicted_class : {predicted_class}')
    # print(f'residuals : {residuals}')
    return predicted_class

In [7]:
# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms_pca:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)

accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

Classification Accuracy: 80.38%


In [8]:
# Calculate confusion matrix
print(f"\nTotal testing channel: {test_labels.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Total testing channel: (3200,)
tp: 1486
tn: 1086
fp: 514
fn: 114
MDR: 0.32125
FAR: 0.07125
AR: 0.80375
